In [1]:
import glob
import sys
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
sys.path.append("../alt_taggers")
from modules.models.part import ParticleTransformer, PartTrainer
from utils.data_loader import DataLoader

In [4]:
# from alt_taggers.modules.models.part import ParticleTransformer, PartTrainer
# from alt_taggers.utils.data_loader import DataLoader

In [5]:
OUT_CONVERTED_PATH = "../../../Datalake/converted/"
BS = 2
model = ParticleTransformer(num_classes = 6,
                 num_enc = 8,
                 num_head = 8,
                 embed_dim = 128,
                 cpf_dim = 16,
                 npf_dim = 8,
                 vtx_dim = 14,
                 for_inference = False,)
model.to(device);

In [6]:
train_loader = DataLoader(OUT_CONVERTED_PATH, BS).get_loader()
val_loader = DataLoader(OUT_CONVERTED_PATH, BS).get_loader()

In [7]:
num_epochs = 10
lr_epochs = max(1, int(num_epochs * 0.3))
lr_rate = 0.01 ** (1.0 / lr_epochs)
mil = list(range(num_epochs - lr_epochs, num_epochs))
print(lr_rate)
print(mil)

0.2154434690031884
[7, 8, 9]


In [8]:
from alt_taggers.utils.pt_ranger import Ranger


In [9]:
optimizer = Ranger(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=mil, gamma=lr_rate)

In [10]:
trainer = PartTrainer(
    optimizer=optimizer,
    scheduler=scheduler,
    model=model,
    device=device)

In [11]:
loss_f = torch.nn.CrossEntropyLoss()

In [12]:
def cross_entropy_one_hot(input, target):
    return torch.nn.CrossEntropyLoss()(input, labels)


In [14]:
trainer.train(num_epochs, train_loader, val_loader, path="/home/jose/Documents/WORKS/CERN/InProgress/OwnArch/saved_models")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:  0


  0%|          | 0/1683 [00:00<?, ?it/s]

  0%|          | 0/1683 [00:00<?, ?it/s]